# Airflow в production. Масштабирование

Многие IT-компании используют Airflow ежедневно в своей работе, а для многих Дата инжинеров Airflow - один из основных инструментов. Познакомившись с деталями и особенностями написания пайплайнов данных на Airflow, рассмотрим, как все это выглядит в продакшене, как происходит масштабирование и разберем пример реальной рабочей задачи с применением этого инструмента.

Отметим различия в специфике работы Data Engineer в контексте разных стеков инструментов. Какие-то компании предпочитают использовать облачные решения и арендуют мощности, о подобных стеках больше рассказывают на фак-те [Cloud Data Engineering](https://new.geekbrains.ru/cloud-data-engineering). В этих случаях можно встретить системы на подобие такой:

<img src="images/4/prod_1.png" style="width: 500px;">

Другие же, имеют свои физические мощности и развертывают кластера на своих серверах. Так может выглядеть архитектура решения с применением кластера [Kubernetes](https://kubernetes.io/): 

<img src="images/4/prod_2.png" style="width: 600px;">

Если объективной надобности в кластере нет, так как команда Дата инжинеров небольшая, нагрузки не велики и нет распределенных вычислений на Spark, к примеру, а при этом Airflow пользоваться хочется, то можно собрать такое решение, которое будет так же прекрасно покрывать потребности конечных пользователей:

<img src="images/4/gamestat.png" style="width: 600px;">

Очевидно, что вариантов архитектуры систем для работы с данными практически столько же, сколько существует компаний, так как у каждой свои условия и особенности. Разберем детальней одну из распространенных связок, а именно __Airflow + Kubernetes__.

## Airflow + Kubernetes

<img src="images/4/airflow_k8s_pods.png" style="width: 400px;">

Для взаимодействия с [Kubernetes](https://airflow.apache.org/docs/apache-airflow/stable/kubernetes.html) (a.k.a. __k8s__) в Airflow есть специальные __KubernetesExecutor__ и __KubernetesPodOperator__, которые позволяют выполнять задачи DAG-графа в среде Kubernetes. Есть два способа, как можно запустить Airflow DAG в Kubernetes:

* с помощью __KubernetesExecutor__, создавая новый pod для каждого экземпляра задачи, в этом случае можно самостоятельно распределить ресурсы, загружая зависимости в Docker-контейнеры, в таком случае Airflow кластер становится динамичным, не тратя ресурсы на не используемые узлы, в отличие от CeleryExecutor;
* использовать __KubernetesPodOperator__, который выполняет конкретную задачу в модуле (pod) внешнего кластера Kubernetes, это позволяет развертывать произвольные Docker-образы, снижая взаимные зависимости между контейнерами.

Модуль pod — это минимальная единица развертывания в Kubernetes. Этот объект инкапсулирует один или несколько контейнеров с приложениями, ресурсы хранения (общие тома), уникальный сетевой IP-адрес и параметры, определяющие порядок работы контейнеров. Как правило, Kubertenes Pod запускает один Docker-контейнер, соответствующий компоненту конкретного приложения. Таким образом, в связке Airflow и Kubernetes, K8s можно рассматривать как пул ресурсов, дающий простой, но мощный API для динамического запуска сложных развертываний.

### KubernetesExecutor

Учитывая основное назначение K8s, можно сделать вывод, что [KubernetesExecutor](https://airflow.apache.org/docs/apache-airflow/stable/executor/kubernetes.html) решает главную проблему Apache Airflow — динамическое распределение ресурсов. До появления Kubernetes Executor все предыдущие решения включали статические кластеры worker’ов. Поэтому Data Engineer или DevOps-инженер должен был заранее определить, какой размер кластера ему необходим в соответствии с возможными рабочими нагрузками. Такая политика могла привести к чрезмерной или недостаточной подготовке кластера, к потере ресурсов или к снижению производительности. Кроме того, требовалась настройка всех зависимостей worker’ов Airflow, чтобы работать с разнообразными заданиями. В частности, для использования другого исполнителя Airflow - CeleryExecutor - необходимо несколько дополнительных технологий (Celery, RabbitMQ, Redis, Flower и пр.), которые придется контролировать.

Таким образом, KubernetesExecutor дает следующие преимущества работы с Apache Airflow:

* __высокий уровень гибкости__, когда кластер Airflow динамически масштабируется в зависимости от рабочей нагрузки, позволит избежать нехватки ресурсов или простаивающих узлов;
* __конфигурация pod'ов на уровне задач__, поскольку KubernetesExecutor создает новый pod для каждого экземпляра задачи, можно точно указать необходимые для конкретного модуля ресурсы (процессор, память и Docker-образ c нужными зависимостями);
* __отказоустойчивость__, благодаря изоляции задачи в отдельном pod, в случае сбоя она не приведет к выходу из строя целого worker’а Airflow, а при отказе планировщика версионирование ресурсов (функция «resourceVersion») в Kubernetes, позволит быстро вернуться в рабочее состояние;
* __упрощенное развертывание__, когда можно указать все параметры в одном файле YAML, а зависимости выгружаются в Docker-контейнеры.

Поскольку по умолчанию Airflow использует только одну папку с файлами DAG, возникает вопрос, как распределить их в кластере K8s. KubernetesExecutor позволяет сделать это тремя способами:
* использовать режим Git-init, клонируя Git с Docker-контейнером при инициализации каждого пода;
* работать в постоянном режиме, смонтировав том с DAG-файлами;
* предварительно подготовить Docker-образ с DAG-файлами (режим «pre-bake»).

Обычно режимы Git-init и «pre-bake» рекомендуются для разработки и небольших кластеров Airflow, например, с менее 1000 заданий. Это обусловлено отсутствием распределенных файловых систем в указанных случаях. В свою очередь, режим постоянного тома рекомендуется для больших папок со множеством DAG-файлов.

Такой подход немного меняет архитектурную схему, которую мы рассматривали ранее:
<img src="images/4/arch-diag-kubernetes.png" style="width: 800px;">

### KubernetesPodOperator

При создании DAG для отправки задания (job) в Apache Spark или определения собственной функции на языке Python, пользователь Airflow будет использовать оператор, например, SparkSubmitOperator или PythonOperator соответственно. По умолчанию фреймворк включает набор встроенных операторов для Apache Spark, Hive, BigQuery и Amazon EMR. Также этот batch-фреймворк позволяет Data Engineer разрабатывать свои собственные операторы и коннекторы с помощью REST-API и специальных плагинов.

Тем не менее, уже готовый [KubernetesPodOperator](https://airflow.apache.org/docs/apache-airflow-providers-cncf-kubernetes/stable/operators.html#howto-operator-kubernetespodoperator) предоставляет пользователям следующие преимущества:

* __простота развертывания__, теперь разработчику нет необходимости создавать новый плагин при генерации нового оператора, любая задача Airflow, которая может быть запущена в контейнере Docker, доступна через уже существующий оператор без дополнительного кода;
* __гибкость конфигурирования и настройки зависимостей__, пользовательские образы Docker гарантируют идемпотентность среды исполнения задач, конфигурации и всех зависимостей, поэтому управление зависимости становится значительно проще; например, если необходимо выполнить одну задачу с использованием SciPy, а другую – на базе NumPy, Дата инжинеру теперь не придется сохранять обе зависимости во всех worker’ах Airflow или переносить задачу на внешнюю узел;
* __информационная безопасность__, Airflow Kubernetes Operator позволяет использовать средства обеспечения cybersecurity от K8s, в частности, пользователи Airflow могут применять технологию Kubernetes Vault для хранения всех конфиденциальных данных, чтобы изолировать любые ключи API, пароли и учетные данные для входа в систему; благодаря этому worker’ы Airflow не будут иметь доступа к такой информации, а могут лишь запросить, чтобы участки data pipeline’ов работали только с теми данными, которые действительно необходимы и достаточны.

Схематично KubernetesPodOperator работает по следующему принципу:

<img src="images/4/k8s_pod_operator.png" style="width: 500px;">

1. Для генерации запроса, который обрабатывает API-server, используется Python-клиент Kubernetes;
2. Kubernetes запускает пользовательский модуль (pod) с любыми заданными характеристиками;
3. Всего одна команда потребуется для загрузки образов со всеми необходимыми переменными среды, секретными данными и зависимостями; после запуска задания оператору необходимо только следить за исправностью журналов отслеживания (логами), у пользователя Airflow будет возможность собирать журналы локально для планировщика или другой распределенной службы ведения журналов, развернутой в кластере Kubernetes.

При использовании KubernetesPodOperator можно развернуть Airflow в конфигурации с Celery на pod'ах кластера k8s, в каждом pod запустить нужный компонент и создать несколько worker'ов для запуска pod операторов. Так может выглядеть такая конфигурация на платформе [Rancher](https://rancher.com/) для управления кластером Kubernetes:

<img src="images/4/rancher.png" style="width: 1000px;">

Обычно для запуска Airflow в кластере Kubernetes используют [надежный docker-образ](https://hub.docker.com/r/puckel/docker-airflow), который собирается автоматически и содержит entrypoint-скрипт. Он нужен, чтобы контейнер мог работать в роли планировщика, веб-сервера или обработчика (воркера) задач и т.д. Также можно [самостоятельно](https://ealebed.github.io/posts/2020/%D1%80%D0%B0%D0%B7%D0%B2%D0%B5%D1%80%D1%82%D1%8B%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5-apache-airflow-%D0%B2-kubernetes/) создать нужный docker-образ для своего production-окружения .

Чтобы запустить собственный data pipeline на Airflow, развернутом в кластере Kubernetes, можно добавить DAG-файлы в docker-образ во время его сборки. Однако, при любом изменении цепочки задач (DAG’а) придется собирать этот образ заново. Вместо этого можно хранить DAG-файлы на внешнем томе, монтируя его в соответствующие pod'ы (scheduler, webserver, worker) при запуске, но более оптимальным считается использовать отдельный контейнер __git-sync__ в pod'е k8s для периодической синхронизации DAG-файлов с указанным git-репозиторием без перезапуска самого Airflow.

Стоит также отметить, что существует еще оператор Airflow для k8s от [Google Cloud Platform](https://github.com/GoogleCloudPlatform/airflow-operator). Данное решение Google не поддерживает официально, а лишь публикует его альфа-версии на GitHub. При этом оператор определяется как контроллер приложения, который расширяет API Kubernetes для создания, настройки и управления приложениями с отслеживанием состояния, инкапсулируя область действия приложения и ежедневные операции. API оператора реализуется через расширение существующего API Kubernetes определениями пользовательских ресурсов (CRD, Custom Resources Definitions), которые декларативно описывают намерение. Эти ресурсы сериализуются как json и хранятся на сервере API. Наблюдая за CRD, контроллер выполняет управляющие действия для перевода системы в желаемое состояние. Благодаря этому платформа K8s пригодна для сложных приложений с сохранением состоянием (stateful).

Таким образом, такой оператор – это совокупность API K8s с CRD (декларативная спецификация) и пользовательским контроллером. Все это необходимо, чтобы эффективно использовать k8s для развертывания Airflow.

## Пример рабочей задачи в связке Airflow с Celery в k8s + Spark + GitLab

Популярный фреймворк распределенной обработки больших данных __Apache Spark__ часто можно видеть в связке с Airflow. Задача обработки данных на Spark - job'а - обычно отправляется на кластер Hadoop для расчета по средством _spark-submit_. В Airflow есть оператор для этой цели - SparkSubmitOperator, однако, если Airflow работает в связке с Kubernetes, то можно команду spark-submit запустить в рамках оператора KubernetesPodOperator. Рассмотрим _реальную задачу Data Engineer в production_.

1. __Situation__: вы Data Engineer в крупной компании, владеющей петабайтами данных; стэк компании: GitLab, Airflow, Hadoop, Kubernetes, Spark.
2. __Target__: ваша задача написать пайплайн обработки данных на Spark и поставить его на расписание в Airflow, это может быть сборка витрины, обогащенной из разных источников.
3. __Action__: выполним необходимую последовательность действий, позволяющую достигнуть цели (подробнее ниже).
4. __Result__: получили автономный процесс сборки витрины и автонобновление ее по расписанию.

Разберем самый важный пункт из описанной последовательности - Action. Распишем костяк кода, который необходимо написать для этой задачи.

1. Для начала напишем \*.py файл с задачей Spark'а, которую будем сабмитить на кластер Hadoop.

In [ ]:
# all imports

def main():
    spark = (
        SparkSession
        .builder
        .appName(APP_NAME)
        .enableHiveSupport()
    ).getOrCreate()
        
    result = (
        spark
        .table('table_name')
        .filter(F.col('date').between(start_date, end_date))
        .withColumnRenamed('col_name', 'col_new_name')
        .select('col_1', 'col_2', 'col_new_name')
    )
    
    result.write \
    .format("orc") \
    .mode("overwrite") \
    .saveAsTable('table_name_result')
    
if __name__ == '__main__':
    main()

2. Далее напишем DAG файл.

In [ ]:
from airflow.contrib.operators.kubernetes_pod_operator import KubernetesPodOperator
from airflow.contrib.kubernetes.volume_mount import VolumeMount
from airflow.contrib.kubernetes.volume import Volume
from airflow.contrib.kubernetes.secret import Secret
from airflow.models import DAG
# other imports

default_args = {
    'owner': 'you',
    'start_date': airflow.utils.dates.days_ago(2),
    'retry_delay': timedelta(minutes=1),
    'retries': 1,
    'params': {'spark_dependencies_subpath': '/folder/'}
}

with DAG(
        'dag_id',
        max_active_runs=10,
        schedule_interval=None,
        catchup=False,
        default_args=default_args
) as dag:
    pod_operator_kwarg = dict(labels={"app": "project"},
                              image_pull_secrets='gitlab-deploy-token',
                              namespace='project-airflow',
                              image='path/to/image/spark:2.6.5-0.5.5',
                              do_xcom_push=False,
                              in_cluster=True,
                              get_logs=True,
                              service_account_name="airflow",
                              is_delete_operator_pod=True,
                              volumes=[Volume(name='project',
                                              configs={'persistentVolumeClaim': {'claimName': 'airflow'}})],
                              volume_mounts=[VolumeMount(name='project',
                                                         mount_path='/mnt/airflow/spark_dependencies',
                                                         sub_path='project/spark_dependencies/' + 
                                                                     dag.params['spark_dependencies_subpath'],
                                                         read_only=True)],
                              secrets=[Secret(deploy_type='env',
                                              deploy_target='KEYTAB_PRINCIPAL',
                                              secret='airflow-krb5',
                                              key='principal')],
                              resources={'request_memory': '512Mi',
                                        'request_cpu': '0.5',
                                        'limit_memory': '1G',
                                        'limit_cpu': '1'},
                              dag=dag)

    args = dict(application='/mnt/airflow/spark_dependencies/project/jobs/run_job.py',            
                keytab='/etc/file.keytab',
                principal='$KEYTAB_PRINCIPAL',
                application='application')

    my_task = KubernetesPodOperator(
        cmds=['bash', '-cx', f'spark-submit {" ".join(args)}'],
        task_id='project_task',
        **pod_operator_kwarg
    )

3. Данный код лежит на GitLab, весь проект на Python, описан также [YAML-файл](https://docs.gitlab.com/ee/ci/yaml/gitlab_ci_yaml.html), который содержит инструкции по сборке и запуску [unit-тестов](https://habr.com/ru/company/yandex/blog/517266/) проекта, какие-то еще инструкции, а также загрузка DAG'ов в Airflow.

4. В GitLab UI запускаем загрузку DAG'ов в Airflow (stage upload-dags-dev), после этого DAG будет виден в Airflow и его можно запускать. Готово!

## Новое в Airflow 2.0

В рамках данного курса мы рассматривали свежую версию Airflow 2.0, но в части компаний какое-то время еще будут работать предыдущие версии. В версии 2.0 было [представлено](https://www.astronomer.io/blog/introducing-airflow-2-0) много важных нововведений, поэтому нужно разобрать основные отличия данной версии. Airflow также [дает советы](https://airflow.apache.org/docs/apache-airflow/stable/upgrading-to-2.html) по переходу на версию 2.0.

1. __Новый scheduler с Low-Latency и High-Availability (HA)__
    * Celery и k8s executors позволяли масштабировать кол-во тасок, но узким местом оставался scheduler
    * Можно запускать реплики scheduler'а (горизонтальное масштабирование)
    * Снижена задержка запуска тасок даже на одном scheduler'е
    * С 2+ scheduler'ами Recovery Time тасок будет нулевым
    * Для работы нескольких scheduler'ов требуется многопоточная работа с мета-базой, поэтому есть ограничения на используемые БД (PostgreSQL 9.6+, MySQL 8+), иначе будут ошибки с SKIP LOCKED и NOWAIT SQL
2. __Полноценный REST API__
    * Прежний API был "нестабилен" и лишен множества полезных возможностей
    * Запускать выполнение DAG'ов программно все так же, как и раньше, но добавилось много доп. функционала
3. __Улучшение Sensors__
    * Оптимизировано использование слотов и потребление CPU/RAM
4. __[TaskFlow API](https://airflow.apache.org/docs/apache-airflow/stable/tutorial_taskflow_api.html) для задания DAG__
    * Упрощен процесс передачи данных между тасками (XCom)
    * Создание PythonOperator прямо из Python функций с помощью декоратора
    * Зависимости между тасками задаются через return функций и аргументы
    * Кастомный бэкенд XCom (S3, HDFS)
5. __Task Groups__
    * Группировка тасок в UI
6. __Независимые providers__
    * Операторы внешних систем (AWS, Azure, Slack etc.) отделены от основного функционала Airflow и помещены в отдельную директорию
7. __Упрощены Kubernetes Executor и KubernetesPodOperator__
    * Удалили 3000 строк исходного кода, работа с k8s стала проще, понятнее и быстрее
8. __Улучшения в UI/UX__
    * Более 30 обновлений, в т.ч. автообновление статусов тасок (Auto-refresh)
9. __[DAG Serialization](https://airflow.apache.org/docs/apache-airflow/stable/dag-serialization.html)__
    * Раньше scheduler и webserver оба парсили DAG файлы, теперь их парсит только scheduler, сериализует их в JSON и сохраняет в Metadata DB, а webserver считывает их оттуда
    <img src="images/4/dag_serialization.png" style="width: 900px;">

Благодаря TaskFlow API, пайплайны можно писать так:

In [ ]:
from airflow.decorators import dag, task
from airflow.utils.dates import days_ago


@dag(default_args={'owner': 'airflow'}, schedule_interval=None, start_date=days_ago(2))
def tutorial_taskflow_api_etl():
    @task
    def extract():
        return {"1001": 301.27, "1002": 433.21, "1003": 502.22}

    @task
    def transform(order_data_dict: dict) -> dict:
        total_order_value = 0

        for value in order_data_dict.values():
            total_order_value += value

        return {"total_order_value": total_order_value}

    @task()
    def load(total_order_value: float):
        print("Total order value is: %.2f" % total_order_value)

    order_data = extract()
    order_summary = transform(order_data)
    load(order_summary["total_order_value"])

tutorial_etl_dag = tutorial_taskflow_api_etl()

## Cложности и ограничения Airflow

В первом уроке мы подробно рассмотрели сильные стороны Airflow, однако, как и у любого инструмента, у Airflow есть несколько слабых сторон и ограничений, среди них:

* наличие __неявных зависимостей__ при установке, например, дополнительные пакеты типа greenlet, gevent, cryptography и пр. усложняют быстрое конфигурирование этого фреймворка, в продакшене добавляются зависимости с Celery, Redis/RabbitMQ, PostgreSQL
* большие __накладные расходы__ (временная задержка 5-10 секунд) на постановку DAG’ов в очередь и приоритезицию задач при запуске, хотя эту проблему в целом решили в версии 2.0 благодаря Scheduler High Availability
* необходимость наличия __свободного слота__ в пуле задач и __рабочего экземпляра планировщика__, например, сенсоры бывает выделяют в отдельный пул, чтобы таким образом контролировать количество и приоретизировать их, сокращая общую временную задержку (latency)
* __пост-фактум оповещения о сбоях__ в конвейере данных, в частности, в интерфейсе Airflow логи появятся только после того, как задание, к примеру, Spark-job, отработано, поэтому следить в режиме онлайн, как выполняется пайплайн, приходится из других мест, например, веб-интерфейса YARN (All Application)
* __разные контексты операторов__ — каждый оператор Airflow исполняется в своем python-интерпретаторе; файл, который создается для определения DAG – это не просто скрипт, который обрабатывает какие-то данные, а объект; в процессе выполнения задачи DAG'а не могут пересекаться, так как они выполняются на разных объектах и в разное время, при этом на практике иногда возникает потребности, чтобы несколько операторов Airflow могли выполняться в одном Spark-контексте над общим пространством dataframe’ов; реализовать это можно с помощью Apache Livy – REST-API сервиса для взаимодействия с кластером Spark

## Альтернативы Airflow

### Luigi

Наиболее известной альтернативой Apache Airflow можно назвать __Luigi__ – библиотеку Python, которую можно установить с помощью инструментов управления пакетами, таких как pip и conda. Luigi также предназначен для управления рабочими процессами посредством их визуализации в виде DAG-конвейера. Этот фреймворк проще в эксплуатации, чем Airflow, но имеет меньше функций и больше ограничений.

<img src="images/4/luigi_ui.png" style="width: 1000px;">

Luigi был создан в Spotify для запуска сложных конвейеров в рекомендательной системе на базе Apache Hive, Spark и других технологий Big Data. В качестве open-source проекта под лицензией Apache 2.0 Luigi стал доступен в 2012 году. Эта система представляет собой Python-пакет, который позволяет распараллеливать рабочие процессы, но, в отличие от Airflow, в Luigi нет планировщика для запуска задач по расписанию, поэтому пользователям приходится полагаться на cron для планирования заданий.

Хотя Airflow и Luigi имеют разные функции, у них много общего:

* наличие наглядного веб-GUI для визуализации конвейеров обработки данных;
* использование Python в качестве языка описания DAG;
* статус [open-source](https://github.com/spotify/luigi) проекта и отсутствие платы за использование;
* пакетный характер работы с данными (batch processing);
* популярность в области Big Data среди известных компаний. 

Luigi применяется в Spotify, Okko, Deloitte, ЦИАН. Разговор про отличия Airflow и Luigi, в основном, сводится к перечислению функциональных возможностей, которые есть у первого и отсутствуют у второго. Такие ограничения считаются недостатками Apache Luigi, о чем мы поговорим далее.

Ключевыми минусами Apache Luigi, которые особенно значимы в практической детальности инженера Big Data, можно назвать следующие:

* отсутствие механизма запуска задач по расписанию, что вызывает необходимость использования crontab, однако, в Luigi имеется центральный планировщик и возможности использования календаря, что в ряде случаев дает большую гибкость по сравнению с Airflow, где нет календарного расписания;
* трудности масштабирования из-за слишком тесной связи DAG-задач с cron-заданиями, что ограничивает количество рабочих процессов, Luigi не может автоматически распределять задачи между worker’ами на разных узлах подобно Celery в Airflow, используя единый брокер сообщений, такой как, Redis или RabbitMQ;
* неудобство GUI – Luigi, в отличие от Airflow, не позволяет пользователям просматривать несколько задач DAG перед выполнением конвейера,  целом интерфейс Luigi выглядит аскетичным и «запутанным» по сравнению с Airflow, в частности, чтобы просмотреть логи задач и исполнение кода, нужно потратить достаточно времени на поиск этих данных; также нельзя получить метаинформацию о выполняемых задачах, узнать сведения по запускаемым задачам и процессам обработки данных; нет способа, например, получить общий список задач, которые зависят от данной задачи; по сути, веб-интерфейс планировщика позволяет только увидеть, почему тот или иной набор задач не выполняется;
* перезапуск конвейеров данных невозможен, хотя если в одной из задач произошла ошибка, DAG восстановится самостоятельно, не нужно перезапускать весь конвейер снова, а уведомить о возникновении ошибок Luigi может email-сообщением.
* отсутствие предварительной проверки выполнения задачи, проверка того, выполнена ли задача, происходит только во время построения графа зависимостей, поэтому при запуске pipeline’а чаще, чем общее время его выполнения, может возникнуть ситуация, что запущены две одинаковые задачи.


### Argo, KubeFlow, MLFlow

Если Airflow и Luigi воплощают идеи _DataOps_, облегчая работу инженера данных с конвейерами их обработки, то __Argo__ и __KubeFlow__ ближе к DevOps-инструментам, т.к. тесно связаны с технологиями контейнеризации. В частности, Argo представляет собой расширение Kubernetes, где каждая задача запускается как отдельный модуль этой платформы контейнеризации. В отличие от Airflow и Luigi, Argo использует YAML, а не Python для определения DAG-задач. Argo предназначен для оркестровки любых задач в экосистеме Kubernetes. Основанный на нем Kubeflow фокусируется на операциях машинного обучения, таких как отслеживание экспериментов, настройка гиперпараметров и развертыванию модели Machine Learning в production. Еще одним отличием Kubeflow от Argo является язык определения задач: в Argo это YAML, а в Kubeflow – Python.

Kubeflow, как и еще одно решение - __MLFlow__, можно отнести к инструментарию _MLOps_ для комплексного и автоматизированного управления жизненным циклом систем Machine Learning. При этом Kubeflow полагается на Kubernetes, а MLFlow – это библиотека Python, которая помогает добавить отслеживание экспериментов в существующий код машинного обучения. Kubeflow позволяет создать полноценный DAG, каждая задача которого представляет собой модуль Kubernetes. MLFlow имеет встроенные функции для развертывания ML-моделей scikit-learn в Amazon Sagemaker или Azure ML.

Сравнение популярности Airflow с аналогами:

<img src="images/4/tools_comparison.png" style="width: 500px;">

### Summary

Набирающая популярность контейниризация повлияла и на Airflow, этот инструмент все чаще можно встретить в конфигурации с Kubernetes. Плюсы такого подхода безусловно стоят усилий по развертыванию всех необходимых сервисов. В случае больших объемов данных в pod'ах часто запускают Spark job'ы на кластере.

Вместе с новой версией Airflow 2.0, запуск DAG'ов в pod'ах k8s позволяет не только максимально рационально использовать ресурсы, но и иметь минимальную задержку между тасками. TaskFlow API упростило написание пайплайнов, особенно тех, в которых требовались XComs.

Airflow имеет ряд интересных для рассмотрения аналогов, но на текущий момент является самым распространным инструментом для разработки пайплайнов данных и одним из основных инструментов Data Engineer.

## Дополнительные материалы

1. [Airflow – платформа для разработки, планирования и мониторинга рабочих процессов](https://www.youtube.com/watch?v=A6YuWmwaTSw)
2. [Production Deployment](https://airflow.apache.org/docs/apache-airflow/stable/production-deployment.html)
3. [Best Practices](https://airflow.apache.org/docs/apache-airflow/stable/best-practices.html)
4. [Airflow FAQ](https://airflow.apache.org/docs/apache-airflow/stable/faq.html)
5. [7 достоинств и 5 недостатков Apache AirFlow](https://medium.com/@bigdataschool/7-достоинств-и-5-недостатков-apache-airflow-39fbbc80e702)
6. [Apache Airflow: автоматизация сбора ежедневных вложений электронной почты](https://www.machinelearningmastery.ru/apache-airflow-automating-the-collection-of-daily-email-attachments-213bc7128d3a/)
7. [Quick guide: How to run Apache Airflow with docker-compose](https://medium.com/@xnuinside/quick-guide-how-to-run-apache-airflow-cluster-in-docker-compose-615eb8abd67a)
8. [Airflow vs. Luigi: Which ETL Tool is the Best? ](https://www.xplenty.com/blog/airflow-vs-luigi/)
9. [Строим Data Pipeline на Python и Luigi](https://khashtamov.com/ru/data-pipeline-luigi-python/)
10. [How to Setup Airflow Multi-Node Cluster with Celery & RabbitMQ ](https://medium.com/@khatri_chetan/how-to-setup-airflow-multi-node-cluster-with-celery-rabbitmq-cfde7756bb6a)
11. [Обзор фреймворка Luigi для построения последовательностей выполнения задач ](https://habr.com/ru/company/otus/blog/339904/)
12. [Airflow vs. Luigi ](https://www.astronomer.io/guides/airflow-vs-luigi)
13. [AirFlow KubernetesExecutor: 3 способа запуска и 4 главных плюса для DevOps-инженера](https://medium.com/@bigdataschool/airflow-kubernetesexecutor-3-%D1%81%D0%BF%D0%BE%D1%81%D0%BE%D0%B1%D0%B0-%D0%B7%D0%B0%D0%BF%D1%83%D1%81%D0%BA%D0%B0-%D0%B8-4-%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D1%8B%D1%85-%D0%BF%D0%BB%D1%8E%D1%81%D0%B0-%D0%B4%D0%BB%D1%8F-devops-%D0%B8%D0%BD%D0%B6%D0%B5%D0%BD%D0%B5%D1%80%D0%B0-cefd3acc833e)

## Домашнее задание

Перепишите пайплайн из предыдущего д/з с использованием новой фичи Airflow 2.0 - [TaskFlow API](https://airflow.apache.org/docs/apache-airflow/stable/tutorial_taskflow_api.html).

__Формат сдачи д/з__: приложите ссылку на ваш Git с кодом DAG'а.